In [17]:
import socket
import spiceypy as spice
import time

In [19]:
spice.furnsh("E:/SPICE/naif0012.tls")
spice.furnsh("E:/SPICE/pck00010.tpc")
spice.furnsh("E:/SPICE/earth_000101_250227_241201.bpc")
spice.furnsh("E:/SPICE/moon_pa_de421_1900-2050.bpc")
spice.furnsh("E:/SPICE/de430.bsp") 

In [21]:
HOST = '127.0.0.1'
PORT = 12345
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

print(f"TCP Sunucusu {HOST}:{PORT} adresinde dinleniyor...")
conn, addr = server_socket.accept()
print(f"Bağlantı sağlandı: {addr}")

TCP Sunucusu 127.0.0.1:12345 adresinde dinleniyor...
Bağlantı sağlandı: ('127.0.0.1', 58514)


In [22]:
start_time = spice.str2et("2000-06-21T18:45:00") 
interval = 0.25 
duration = 1000

current_time = start_time
end_time = start_time + duration
next_send_time = time.perf_counter()

In [23]:
try:
    while current_time <= end_time:
        telemetry = []
        now = time.perf_counter()

        if now >= next_send_time:
            
            state_earth, light_earth = spice.spkgeo(targ=399, et=current_time, ref="ECLIPJ2000", obs=10)
            state_moon, light_moon = spice.spkgeo(targ=301, et=current_time, ref="ECLIPJ2000", obs=10) 
        
            rotation_earth = spice.pxform("IAU_EARTH", "ECLIPJ2000", current_time)
            rotation_moon = spice.pxform("IAU_MOON", "ECLIPJ2000", current_time)
            pos_earth = state_earth[:3]
            pos_moon = state_moon[:3]
                
            roll1, pitch1, yaw1 = spice.m2eul(rotation_earth, 1, 2, 3)
            roll_earth = spice.convrt(roll1, "RADIANS", "DEGREES")
            pitch_earth = spice.convrt(pitch1, "RADIANS", "DEGREES")
            yaw_earth = spice.convrt(yaw1, "RADIANS", "DEGREES")
            rot_earth = [roll_earth,pitch_earth,yaw_earth]
        
            roll2, pitch2, yaw2 = spice.m2eul(rotation_moon, 1, 2, 3)
            roll_moon = spice.convrt(roll2, "RADIANS", "DEGREES")
            pitch_moon = spice.convrt(pitch2, "RADIANS", "DEGREES")
            yaw_moon = spice.convrt(yaw2, "RADIANS", "DEGREES")
            rot_moon = [roll_moon,pitch_moon,yaw_moon]
        
            utc_time = spice.et2utc(current_time, "C", 3)
                
            print(f"{pos_earth[0]:.3f},{pos_earth[1]:.3f},{pos_earth[2]:.3f},{rot_earth[0]:.3f},{rot_earth[1]:.3f},{rot_earth[2]:.3f},{pos_moon[0]:.3f},{pos_moon[1]:.3f},{pos_moon[2]:.3f},{rot_moon[0]:.3f},{rot_moon[1]:.3f},{rot_moon[2]:.3f}")
            telemetry = (f"{pos_earth[0]:.3f},{pos_earth[1]:.3f},{pos_earth[2]:.3f},{rot_earth[0]:.3f},{rot_earth[1]:.3f},{rot_earth[2]:.3f},{pos_moon[0]:.3f},{pos_moon[1]:.3f},{pos_moon[2]:.3f},{rot_moon[0]:.3f},{rot_moon[1]:.3f},{rot_moon[2]:.3f}")
            conn.sendall((telemetry + "\n").encode())
                
            next_send_time += interval
            current_time += interval

except KeyboardInterrupt:
    print("Bağlantı kapatılıyor...")

finally:
    conn.close()
    server_socket.close()
    
    spice.unload("E:/SPICE/naif0012.tls")
    spice.unload("E:/SPICE/pck00010.tpc")
    spice.unload("E:/SPICE/earth_000101_250227_241201.bpc")
    spice.unload("E:/SPICE/moon_pa_de421_1900-2050.bpc")
    spice.unload("E:/SPICE/de430.bsp")

1796949.701,-152025579.873,530.538,23.439,-0.003,168.533,2119331.506,-152264830.055,-17220.322,-0.691,1.393,-148.388
1796957.024,-152025579.816,530.538,23.439,-0.003,168.532,2119338.967,-152264829.796,-17220.341,-0.691,1.393,-148.388
1796964.348,-152025579.758,530.538,23.439,-0.003,168.531,2119346.429,-152264829.538,-17220.360,-0.691,1.393,-148.388
1796971.672,-152025579.700,530.538,23.439,-0.003,168.530,2119353.890,-152264829.280,-17220.379,-0.691,1.393,-148.388
1796978.995,-152025579.642,530.539,23.439,-0.003,168.529,2119361.352,-152264829.022,-17220.398,-0.691,1.393,-148.388
1796986.319,-152025579.585,530.539,23.439,-0.003,168.528,2119368.813,-152264828.763,-17220.417,-0.691,1.393,-148.388
1796993.643,-152025579.527,530.539,23.439,-0.003,168.527,2119376.275,-152264828.505,-17220.436,-0.691,1.393,-148.388
1797000.967,-152025579.469,530.539,23.439,-0.003,168.526,2119383.736,-152264828.247,-17220.455,-0.691,1.393,-148.388
1797008.290,-152025579.411,530.540,23.439,-0.003,168.525,2119391

ConnectionAbortedError: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte